# Training a Neural Network with Pytorch

- 5.1: Define a SingleHiddenLayer Network as a pytorch module
- 5.2: Define the cost function
- 5.3: Setup the training function
- 5.4: Setup a validation/testing function
- 5.5: Create a training/validation/testing split of your data
- 5.6: Iterate over your dataset (epoch) and train your network using the train() and validate() methods
- 5.7: Make Predictions on the training and test set and plot the results

In [ ]:
from torch.utils.data import TensorDataset, DataLoader 

## Define a SingleHiddenLayer Network as a pytorch module

In [ ]:
class SingleHiddenLayerNetwork(nn.Module):
    def __init__(self, I, H, O):
        super(SingleHiddenLayerNetwork, self).__init__()
        self.hidden_1 = nn.Linear(I, H, bias=False)
        self.output = nn.Linear(H, O, bias=False)
        self.activation = nn.Sigmoid()
        
    def forward(self, X):
        z1 = self.hidden_1(X)
        a1 = self.activation(z1)
        z2 = self.output(a1)
        a2 = self.activation(z2)
        return a2

## Define the cost function

In [ ]:
def bce_loss(y, a2):
    return -1/y.size(0)*(y*a2.log()+(1-y)*(1-a2).log()).sum(0)

## Setup the training function

In [ ]:
def train(model, optimizer, data_loader):
    model.train()
    for X, y in data_loader:
        optimizer.zero_grad()  # reset gradients
        a2 = model(X)          # forward propagtion through the model
        loss = bce_loss(y, a2) # compute loss
        loss.backward()        # backpropagate
        optimizer.step()       # update model parameters using the gradients
    
    y_pred = np.where(a2[:, 0].detach().numpy()>0.5, 1, 0)
    accuracy = accuracy_score(y, y_pred)
    return loss, accuracy

In [ ]:
Setup a validation/testing function

In [ ]:
def evaluate(model, data_loader):  
    model.eval()
    for X, y in data_loader:
        with torch.no_grad():
            a2 = model(X)
            loss = bce_loss(y, a2)
    y_pred = np.where(a2[:, 0].numpy()>0.5, 1, 0)
    accuracy = accuracy_score(y, y_pred)
    return loss, accuracy

## Create a training/validation/testing split of your data

In [ ]:
#Define the size of the input, hidden, and output layers
I, H, O = 2, 4, 1

#Use Sklearn to create two-moons + noise
X_train, y_train, X_test, y_test = make_train_test(batch_size, batch_num, test_size, noise=0.2)

#Define Train Set in Pytorch
X_train = torch.from_numpy(X_train).float()[0] #Convert to torch tensor, single batch
y_train = torch.from_numpy(y_train).float()[0] #Convert to torch tensor, single batch

train_dataset = TensorDataset(X_train, y_train) # wrapper around dataset that helps DataLoader

#Define Test Set in Pytorch
X_test = torch.from_numpy(X_test).float() #Convert to torch tensor, already single batch
y_test = torch.from_numpy(y_test).float() #Convert to torch tensor, already single batch

test_dataset = TensorDataset(X_test, y_test)

#Use Pytorch's functionality to load data in batches. Here we use full-batch training again.
train_loader = DataLoader(train_dataset, batch_size=X_train.size(0), shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=X_test.size(0), shuffle=False)

 ## Iterate over your dataset (epoch) and train your network using the train() and validate() methods

In [ ]:
network = SingleHiddenLayerNetwork(I, H, O)
optim = torch.optim.SGD(network.parameters(), lr=1) # we can pass network.parameters to the optimiser
                                                    # instead of passing an explicit list (useful for big networks)
for i in range(1000):
    train_loss, train_accuracy = train(network, optim, train_loader)
    test_loss, test_accuracy = evaluate(network, test_loader)
    
    if i % 100 == 0:
        print("Training Loss in epoch "+str(i)+": %1.2f" % train_loss.item())
        print("Training accuracy in epoch "+str(i)+": %1.2f" % train_accuracy)
        print("Test Loss in epoch "+str(i)+": %1.2f" % test_loss.item())
        print("Test accuracy in epoch "+str(i)+": %1.2f" % test_accuracy, "\n")

## Make Predictions on the training and test set and plot the results

In [ ]:
network.eval()   #   tell the network we are in evaluation mode (deactivates mini-batches, dropouts, etc)
with torch.no_grad():    # deactivates the autograd engine (to not safe grads, etc)
    a_train = network(X_train)
    a_test = network(X_test)
print("Test set accuracy: ", accuracy_score(y_test, np.where(a_test[:, 0].numpy()>0.5, 1, 0)))
fig, ax = plt.subplots(1, 2, figsize=(12, 6))
ax[0].scatter(X_train[:, 0], X_train[:, 1], c=np.where(a_train[:, 0].numpy()>0.5, 1, 0))
ax[1].scatter(X_test[:, 0], X_test[:, 1], c=np.where(a_test[:, 0].numpy()>0.5, 1, 0))